# Creating a Custom Container

In [2]:
import uuid
import sagemaker
import boto3

from datetime import datetime

region = boto3.Session().region_name

In [3]:
VERSION = "1"

ENDPOINTS_FOLDER = "version" + VERSION
REPOSITORY_NAME = "penguins"

CONTAINER_VERSION = f"{datetime.now().year}{str(datetime.now().month).zfill(2)}{str(datetime.now().day).zfill(2)}{str(uuid.uuid4())[-4:]}" 
print(f"Container version: {CONTAINER_VERSION}")

Container version: 20230412c950


In [4]:
!aws ecr create-repository --repository-name $REPOSITORY_NAME


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'penguins' already exists in the registry with id '325223348818'


In [5]:
repository = !aws ecr describe-repositories \
    --repository-names $REPOSITORY_NAME \
    --query "repositories[0].repositoryUri"

repository_uri = repository[0][1:-1]
repository = repository_uri[0:repository_uri.index("/")]
print(f"ECR Repository: {repository}/{REPOSITORY_NAME}")

ECR Repository: 325223348818.dkr.ecr.us-east-1.amazonaws.com/penguins


In [16]:
!aws ecr get-login-password --region $region | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

!docker build -t $repository_uri:$CONTAINER_VERSION .
!aws ecr get-login-password | docker login --username AWS --password-stdin $repository
!docker push $repository_uri:$CONTAINER_VERSION

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  14.34kB
Step 1/9 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.4-cpu AS sagemaker
 ---> f0835ba9191e
Step 2/9 : RUN apt-get clean &&     apt-get update -y &&     apt-get install -y --no-install-recommends     libgl1-mesa-glx
 ---> Using cache
 ---> 9ff654725829
Step 3/9 : RUN pip install --upgrade pip
 ---> Using cache
 ---> b2e45c2355f9
Step 4/9 : RUN pip install pandas
 ---> Using cache
 ---> c7080bf2e79b
Step 5/9 : RUN pip install numpy
 ---> Using cache
 ---> 0a34d2b6538f
Step 6/9 : RUN pip install requests
 ---> Using cache
 ---> 9957241f125d
Step 7/9 : RUN pip install tensorflow==2.4
 ---> Using cache
 ---> 7f2b613bcb7d
Step 8/9 : LABEL com.amazonaws.sagemaker.capabilities.m